# HW Assignment 2

Instructions: Implement both PG and an evolutionary algorithm to solve the Open AI Gym Lunar Lander problem, and then apply it to my area of choice, which is chess.

First, we need to do some setup

In [1]:
import torch
import numpy as np
import gym

# Set the device
if torch.cuda.is_available():
    device = "gpu" # 🧮
# elif torch.backends.mps.is_available():
#     device = "mps" # 🧠
else:
    device = "cpu" # 🥺
    
print(f"Using device: {device}")

Using device: cpu


First, we need to write the code for our Policy Gradient function with a baseline (REINFORCE). I'm going to use PyTorch as my neural network library.

I'm going to start with a basic feed forward net for both the network that chooses the policy and the network that learns states' values.

First, the policy network for choosing actions

In [2]:
from torch import nn

class PolicyChoice(nn.Module):
    def __init__(self):
        super(PolicyChoice, self).__init__()
        self.flatten = nn.Flatten()
        self.policy = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4), # TODO: try reducing to one hidden layer if learning proves initially dificult
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.Softmax() # log softmax for a nice interpretation as probabilities of choosing actions
        )
        # self.first_lin = nn.Linear(8, 8)
        # self.first_relu = nn.ReLU()
        # self.second_lin = nn.Linear(8, 4) # TODO: try reducing to one hidden layer if learning proves initially dificult
        # self.second_relu = nn.ReLU()
        # self.third_lin = nn.Linear(4, 4)
        # self.soft_max = nn.Softmax(dim=0) # log softmax for a nice interpretation as probabilities of choosing actions

    def forward(self, x):
        probs = self.policy(x)
        # x = self.first_lin(x)
        # x = self.first_relu(x)
        # x = self.second_lin(x)
        # x = self.second_relu(x)
        # x = self.third_lin(x)
        # x = self.soft_max(x)
        # return x
        return probs

policy_model = PolicyChoice().to(device)
policy_adam = torch.optim.Adam(policy_model.parameters(), 1e-3)

For our loss function for the policy network, we want to adjust just the parameters with the primary aim of affecting the probability of taking the action that we took on that time step. If the return of the resulting state is better than expected, we want to increase it proportionally. If it is less than expected, we want to decrease it proportionally. Thus, we multiply the gradient of the parameter weights w.r.t. the taken action's probability by the difference of the return for that state-action pair.

Importantly, there is an extra factor however that we must consider; when we decide that we want to take the gradient of the parameters w.r.t. a specific action's return, the policy expectancy must be multiplied by the specific action's likelihood to determine the value it contributes to the policy. Thus, we end up with the gradient of the action's probability conditioned on the state and parameters. 

Thus, the general concept of loss to backpropogate in the REINFORCE algorithm is:


$\Large (G_t - \hat{\upsilon}) \frac{\nabla\pi(A_t|S_t, \theta)}{\pi(A_t|S_t, \theta)}$

This can be expressed as:

$\Large (G_t - \hat{\upsilon}) \nabla \ln{\pi(A_t|S_t, \theta)}$


The code below just worries about the loss and not the gradient, as PyTorch provides autograd differntiation.

In [3]:
def policy_loss(prob, state_util_difference):
    return nn.NLLLoss(prob, torch.ones(1)) * state_util_difference

Now, the network for approximating state utililities.

In [4]:
class StateUtility(nn.Module):
    def __init__(self):
        super(StateUtility, self).__init__()
        self.flatten = nn.Flatten()
        self.state_utility = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4), # TODO: try reducing to one hidden layer if learning proves initially dificult
            nn.ReLU(),
            nn.Linear(4, 1), # output a tensor of a scalar value
        )

    def forward(self, x):
        state_utility = self.state_utility(x)
        return state_utility

state_util_model = StateUtility().to(device)
state_util_adam = torch.optim.Adam(state_util_model.parameters(), 1e-3)

For the state utilities network, we just use L1 loss with the gradients of W with respect to state utility.

$\Large (G_t - \hat{\upsilon}(S_t, W)) \nabla \hat{\upsilon}(S_t, W)$

Like above, the code below just worries about the loss and not the gradient, as PyTorch provides autograd differntiation.

In [5]:
def state_util_loss(calculated_state_value, episode_state_value):
    # the overall state value is the input, and the individual state value is our target
    return nn.L1Loss(calculated_state_value, episode_state_value)


We also need a function to calculate a state instance's utility in a given episode.

In [6]:
def calc_ep_state_util():
    """Given an observation and reward from an"""

Let's define our hyperparameters

In [7]:
gamma = .99

Let's load the Lunar Lander environment now

In [8]:
# TODO: use a custom dataloader class and see if speed up

env = gym.make(
    "LunarLander-v2"
)

action_space_seed = np.random.seed(13)

observation, info = env.reset(seed=13)

# index i in the lists below corresponds to the timestep i of the current episode
observations = []
rewards = []
state_utilities = []

for timestep in range(1000):
    action_weights = np.array(policy_model(torch.tensor(observation)).tolist())
    action_array = np.random.multinomial(n=1, pvals=action_weights)
    action = np.argmax(action_array)
    observation, reward, terminated, truncated, info = env.step(action)
    observations.append(observation)
    rewards.append(reward)
    
    if terminated or truncated:
        ep_length = len(observations)
        for timestep in reversed(range(ep_length)):
            terminal = timestep == len(rewards) - 1
            state_utilities[timestep] = rewards[timestep] + (gamma * state_utilities[timestep+1]) if not terminal else rewards[timestep]
            pred_state_utility = state_util_model(torch.tensor(observations[timestep]))
            loss_state_utility = state_util_loss(pred_state_utility, state_utilities[timestep])
            state_util_adam.zero_grad()
            loss_state_utility.backward()
            state_util_adam.step()
        observation, info = env.reset()
        observations, rewards, utilities = []
env.close()

/Users/jhotz/School/cpsc_532j/hm2_assignment/env/lib/python3.10/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/Users/jhotz/School/cpsc_532j/hm2_assignment/env/lib/python3.10/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


IndexError: list assignment index out of range